# Getting Started Guide

In this notebook we demonstrate how to build and design a page with **esparto**.

In [ ]:
# Environment setup
import os
!pip install -Uqq esparto
if os.environ.get("BINDER_SERVICE_HOST"):
    !pip install -Uqq pandas matplotlib seaborn

In [ ]:
import esparto as es
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

## Page Creation

All pages start with a Page object. When a Page is initialised we are allowed to specify a page title, which will 
appear at the top of the page, a navbrand which will be displayed in the header, and several other options which are
detailed in the documentation.


Creating the Page::

In [ ]:
page = es.Page(title="Page Title")

## Content Types

The following content types are supported:

* Markdown Text
* Images
* Pandas DataFrames
* Plots from Matplotlib, Plotly, and Bokeh

When a content object is added to the page it is automatically converted to the matching Esparto Content class.
Additional options may be chosen by explicitly instantiating the object although default settings should be suitable
for most scenarios.

### Markdown

Creating Markdown content from a string:

In [ ]:
page = es.Page()
markdown_text = """
**Markdown** is a *lightweight markup language* for creating formatted text using a plain-text editor. 
**John Gruber** and **Aaron Swartz** created Markdown in 2004 as a markup language that is appealing to human 
readers in its source code form. Markdown is widely used in:

* blogging
* instant messaging
* online forums 
* collaborative software 
* documentation pages
* readme files

----

<small>*From Wikipedia:* [*Markdown*](https://en.wikipedia.org/wiki/Markdown)</small>
"""
page[0] = es.Markdown(markdown_text)
page[0]

Reading the same Markdown from a text file:

In [ ]:
Path("markdown.md").write_text(markdown_text)
page[0] = "markdown.md"
page[0]

### Images

Reading an image from a file path:

In [ ]:
image_path = "my-image.png"
page[0][0][0] = image_path
page.tree()

In [ ]:
image_credit = "Photo by Benjamin Voros for Unsplash"
page[0][0][1] = es.Image(image_path, caption=image_credit)
page[0]

### Pandas DataFrames

Creating a table from a Pandas DataFrame:

In [ ]:
df00 = sns.load_dataset("mpg")
pandas_df = df00.describe().round(2).T[["mean", "std", "50%"]]

page[0] = es.DataFramePd(pandas_df)
page[0]

### Matplotlib Figures

Creating a plot from a Matplotlib figure:

In [ ]:
mpl_fig, ax = plt.subplots()

colors = ["C0", "C1", "C2"]
df00.groupby("origin")["horsepower"].mean().plot.bar(color=colors, rot=0, ax=ax)
ax.set_title("Mean Horsepower by Origin")
ax.set_ylabel("Horsepower")
ax.set_xlabel("Origin")
mpl_fig.tight_layout()
plt.close()  # prevent auto-plotting in notebook

page[0] = es.FigureMpl(mpl_fig)
page[0]

## Page Layout

### Rows and Columns

Pages are arranged in Sections, Rows, and Columns. If fine control over the layout is not required, the specific row
and / or column can be omitted and Esparto will infer the logical structure:

In [ ]:
my_page = es.Page(title="Page Title")
my_page[0] = "Page Content"
my_page.tree()

#### Add Content

Specifying the Section, Row, and Column:

In [ ]:
page = es.Page("Page Title")
page["Section One"]["Row One"]["Column One"] = "Some content"
page["Section One"]["Row One"]["Column Two"] = "More content"
page.tree()

Adding multiple Columns to the same Row using a tuple of dictionaries:

In [ ]:
page = es.Page("Page Title")
page["Section One"]["Row One"] = (
    {"Column One": "Some content"}, 
    {"Column Two": "More content"}
)
page.tree()

#### Update Content

Update content using indexing or object attributes:

In [ ]:
page["Section One"]["Row One"]["Column One"] = image_path
page.section_one.row_one.column_two = image_path
page.tree()

#### Delete Content

Delete content by index or attribute name:

In [ ]:
del page.section_one.row_one[-1]
page.tree()

Alternatively, we can delete content by accessing its attribute:
```python
del page.section_one.row_one.new_title
```

#### Auto Layout

We can create a basic page with the methods covered so far:

In [ ]:
my_page = es.Page(title="My Report")
my_image = es.Image("my-image.png", caption=image_credit)
my_page["Words and Images"] = ("markdown.md", my_image)

my_page

Esparto infers the full Page structure without explicit instructions. This code generates the same page as above:

In [ ]:
my_page_by_hand = es.Page(title="My Report", children=[
    es.Section(title="Words and Images", children=[
        es.Row(children=[
            es.Column(children=["markdown.md"]),
            es.Column(children=[my_image])
        ])
    ])
])
my_page_by_hand.tree()

In [ ]:
assert my_page == my_page_by_hand
# True

### Content Cards

Cards are useful for grouping related content:

In [ ]:
data_sample = df00.iloc[:, :4].head(5)
data_types = df00.dtypes.rename("Data Type").to_frame()
card_a = es.Card(title="Data Types", children=[data_types])
card_b = es.Card(title="Summary Stats", children=[mpl_fig, pandas_df])
card_a + card_b

Adding Cards to a Section:

In [ ]:
section_two = es.Section(title="Data Analysis")
section_two[0] = (card_a, card_b)
section_two.tree()

### Card Sections

Card Sections can be used when content should be wrapped in Cards by default:

In [ ]:
section_two = es.CardSection(title="Data Analysis")
section_two[0] = (
    {"Data Types": data_types},
    {"Summary Stats": [mpl_fig, pandas_df]}
)
section_two.tree()

### Using Spacers

Spacers create empty Columns in a Row:

In [ ]:
page = es.Page()
page[0] = ({"Markdown": markdown_text}, es.Spacer())
page[0]

### Page Breaks

Page Breaks indicate the end of a page when printing or converting to PDF:

In [ ]:
my_page["Data Analysis"] = section_two
my_page.children.insert(1, es.PageBreak())
my_page.tree()

## Esparto Options

The options available in `es.options` allow control over how dependencies are source and provisioned, 
as well as the behaviour of certain content types.

```python
>>> es.options?
```
```Options for configuring esparto behaviour and output.

Config options will automatically be loaded if a yaml file is found at
either './esparto-config.yaml' or '~/esparto-data/esparto-config.yaml'.

Attributes:
    dependency_source (str):
        How dependencies should be provisioned: 'cdn' or 'inline'.
    bootstrap_cdn (str):
        Link to Bootstrap CDN. Used if dependency source is 'cdn'.
        Alternative links are available via esparto.bootstrap_cdn_themes.
    bootstrap_css (str):
        Path to Bootstrap CSS file. Used if dependency source is 'inline'.
    esparto_css (str):
        Path to additional CSS file with esparto specific styles.
    jinja_template (str):
        Path to Jinja HTML page template.

    matplotlib: Additional config options for Matplotlib.
    plotly: Additional config options for Plotly.
    bokeh: Additional config options for Bokeh.
```

## Saving your Work

### As a Webpage

In [ ]:
my_page.save_html("my-page.html")

### As a PDF

In [ ]:
my_page.save_pdf("my-page.pdf")